In [ ]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import gc
import pickle
import warnings
import tensorflow as tf
import optuna

warnings.filterwarnings('ignore')

Import dataset pickle file

In [ ]:
def get_pickle(pickle_dir, file_name):
    pickle_dir = os.path.join(os.getcwd(), 'datasets/fsc22/Pickle Files/' + pickle_dir)
    fold_dir = os.path.join(pickle_dir, file_name)
    infile = open(fold_dir, 'rb')
    fold = pickle.load(infile)
    infile.close()

    return fold

In [ ]:
with tf.device('/CPU:0'):
    pickle_dir = input('Enter 5 fold pickle dirname: ')
    
    spect_folds = []
    train_spects = []
    
    for fold in range(5):
        spects_fold = get_pickle(pickle_dir, pickle_dir + '_fold' + str(fold+1))
        train_spects.extend(spects_fold)
        spect_folds.append(spects_fold)

    print(f'len train_spects: {len(train_spects)}')

    train_features_df = pd.DataFrame(train_spects, columns=['feature', 'class'])

    del train_spects

    gc.collect()

    X_train = np.array(train_features_df['feature'].tolist())
    y_train = np.array(train_features_df['class'].tolist())

    print(f'X_train shape: {np.shape(X_train)}')
    print(f'y_train shape: {np.shape(y_train)}')
    
    print(f'len spect_folds: {len(spect_folds)}') # num folds
    print(f'len spect_folds[0]: {len(spect_folds[0])}') # samples in a fold
    print(f'len spect_folds[0][0]: {len(spect_folds[0][0])}') # elements in a sample - should be 2
    print(f'shape spect_folds[0][0][0]: {np.shape(spect_folds[0][0][0])}') # spectrogram shape

In [ ]:
with tf.device('/CPU:0'):
    IMG_SIZE = (X_train.shape[1], X_train.shape[2])
    IMG_SHAPE = IMG_SIZE + (3,)
    print(IMG_SHAPE)
    input_shape = IMG_SHAPE

    num_labels = 26 # Should be changed

Create Model

In [ ]:
def get_model():
    base_model = tf.keras.applications.MobileNetV3Small(
        input_shape=IMG_SHAPE,
        weights=None,
        include_top=False
    )
    
    x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(base_model.output)
    outputs = tf.keras.layers.Dense(num_labels, activation='softmax', name='prediction')(x)

    model = tf.keras.models.Model(base_model.input, outputs, name='MobileNetV3-Small_FSC22')
    
    return model

Draw accuracy loss plot

In [ ]:
def save_plot_accuracy_loss(history, model_name, model_save_dir, f=1):
    epochs = history.epoch
    tr_acc = history.history['accuracy']
    tr_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    run_epochs = len(epochs)

    f = f
    save_epochs = [epochs[i] for i in range(0, run_epochs, f)]
    save_tr_acc = [tr_acc[i] for i in range(0, run_epochs, f)]
    save_tr_loss = [tr_loss[i] for i in range(0, run_epochs, f)]
    save_val_acc = [val_acc[i] for i in range(0, run_epochs, f)]
    save_val_loss = [val_loss[i] for i in range(0, run_epochs, f)]

    # Create a figure and axis
    fig, ax1 = plt.subplots()

    fig.set_figheight(12)
    fig.set_figwidth(24)

    # Plot accuracy lines
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy', color='black')
    ax1.plot(save_epochs, save_tr_acc, color='#800000', marker='o', label='Training Accuracy')
    ax1.plot(save_epochs, save_val_acc, color='#000075', marker='x', label='Validation Accuracy')
    # ax1.set_xticklabels(save_epochs, rotation=90)

    # Create a second y-axis for loss lines
    ax2 = ax1.twinx()  # Share the same x-axis
    ax2.set_ylabel('Loss', color='black')
    ax2.plot(save_epochs, save_tr_loss, color='#3cb44b', marker='s', label='Training Loss')
    ax2.plot(save_epochs, save_val_loss, color='#f58231', marker='^', label='Validation Loss')
    ax2.tick_params(axis='y', labelcolor='black')

    # Add a legend
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='best')

    # Set a title
    plt.title('Accuracy and Loss Over Epochs')

    accuracy_matrices_path = model_save_dir

    curr_datetime = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    filename = f'{model_name.lower()}-training_metrics_plot-{format(curr_datetime)}.png'

    if not os.path.exists(accuracy_matrices_path):
        os.makedirs(accuracy_matrices_path)

    # Save the plot to the specified folder
    destination = os.path.join(accuracy_matrices_path, filename)
    plt.savefig(destination, bbox_inches='tight')

### Hyperparameter tuning

Define search space

In [ ]:
from sklearn.model_selection import train_test_split

X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X_train, y_train, test_size=0.3, random_state=42, shuffle=True, stratify=y_train)

def objective(trial):
    # Sample hyperparameters from the search space
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1)
    optimizer = trial.suggest_categorical("optimizer", ["adam", "sgd"])
    num_epochs = trial.suggest_int("num_epochs", 20, 75)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    opt_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate) if optimizer=='adam' else tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model = get_model()

    # Compile the model
    model.compile(optimizer=opt_optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    earlystopping = tf.keras.callbacks.EarlyStopping(monitor="accuracy", mode="max", patience=10, restore_best_weights=True, verbose=1)

    # Train the model with the given hyperparameters
    model.fit(X_train_hp, y_train_hp, epochs=num_epochs, batch_size=batch_size, callbacks =[earlystopping])

    # Evaluate the model on the test set
    accuracy = model.evaluate(X_test_hp, y_test_hp)[1]

    del model

    gc.collect()

    return accuracy

Run hyperparameter tuning

In [ ]:
# Create a study object
study = optuna.study.create_study(direction="maximize")

# Run the optimization
study.optimize(objective, n_trials=50)

del X_train_hp
del X_test_hp
del y_train_hp
del y_test_hp

gc.collect()

In [ ]:
# Plot the hyperparameters
hyperparameter_optimization_path = os.path.join(os.getcwd(), './metrics/hyperparameter_optimization')

curr_datetime = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
filename = f'mobilenetv3s-hyperparameter_optimization_plot-{format(curr_datetime)}.png'

if not os.path.exists(hyperparameter_optimization_path):
    os.makedirs(hyperparameter_optimization_path)

# Save the plot to the specified folder
destination = os.path.join(hyperparameter_optimization_path, filename)

optuna.visualization.matplotlib.plot_optimization_history(study)
plt.savefig(destination, bbox_inches='tight')

In [ ]:
# best hyperparameters found by our experiments

best_learning_rate = 0.06325014821
best_optimizer = 'sgd'
best_num_epochs = 70
best_batch_size = 16

In [ ]:
# Get the best hyperparameters
best_trial = study.best_trial

# Get the values of the best hyperparameters
best_learning_rate = best_trial.params["learning_rate"]
best_optimizer = best_trial.params["optimizer"] 
best_num_epochs = best_trial.params["num_epochs"]
best_batch_size = best_trial.params["batch_size"]

print(f'best_learning_rate: {best_learning_rate}')
print(f'best_optimizer: {best_optimizer}')
print(f'best_num_epochs: {best_num_epochs}')
print(f'best_batch_size: {best_batch_size}')

### Train model

In [ ]:
with tf.device('/CPU:0'):
    model_name = input('Enter model name: ')

    models_dir = os.path.join(os.getcwd(), 'models')
    model_dir = os.path.join(models_dir, "MobileNetV3-Small")
    model_save_dir = os.path.join(model_dir, model_name)
    model_save_path = os.path.join(model_save_dir, model_name + "_base.h5")


    if not os.path.exists(model_save_dir):
        os.makedirs(model_save_dir)

    model = get_model()
    model.summary()

In [ ]:
def get_train_vaild_data(spects, fold):
    train_spects = []
    valid_spects = None
    
    for i in range(5):
        if i + 1 != fold:
            train_spects.extend(spects[i])
        else:
            valid_spects = spects[i]
    
    train_df = pd.DataFrame(train_spects, columns=['feature', 'class'])
    valid_df = pd.DataFrame(valid_spects, columns=['feature', 'class'])

    del train_spects
    del valid_spects

    gc.collect()

    X_train_cv = np.array(train_df['feature'].tolist())
    y_train_cv = np.array(train_df['class'].tolist())
    
    X_valid_cv = np.array(valid_df['feature'].tolist())
    y_valid_cv = np.array(valid_df['class'].tolist())
    
    return X_train_cv, X_valid_cv, y_train_cv, y_valid_cv


In [ ]:
acc_per_fold = []
loss_per_fold = []

# K-fold Cross Validation model evaluation
fold_no = 1

for fold in range(5):
    base_learning_rate = best_learning_rate

    initial_epochs = best_num_epochs
    batch_size = best_batch_size

    final_learning_rate = 0.0005
    learning_rate_decay_factor = (final_learning_rate / base_learning_rate)**(1/initial_epochs)
    
    X_train_cv, X_valid_cv, y_train_cv, y_valid_cv = get_train_vaild_data(spect_folds, fold_no)
    
    # print(f'X_train_cv shape: {np.shape(X_train_cv)}')
    # print(f'y_train_cv shape: {np.shape(y_train_cv)}')

    # print(f'X_valid_cv shape: {np.shape(X_valid_cv)}')
    # print(f'y_valid_cv shape: {np.shape(y_valid_cv)}')
    
    steps_per_epoch = int(np.shape(X_train_cv)[0]/batch_size)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=base_learning_rate,
                    decay_steps=steps_per_epoch,
                    decay_rate=learning_rate_decay_factor,
                    staircase=True)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule) if best_optimizer=='adam' else tf.keras.optimizers.SGD(learning_rate=lr_schedule)

    model = get_model()
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    earlystopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=10, restore_best_weights=True, verbose=1)

    # Fit data to model
    history = model.fit(X_train_cv, y_train_cv,
                batch_size=batch_size,
                epochs=initial_epochs,
                validation_data=(X_valid_cv, y_valid_cv),
                verbose=1, callbacks =[earlystopping])

    # Generate generalization metrics
    scores = model.evaluate(X_valid_cv, y_valid_cv, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    model_split_save_path = os.path.join(model_save_dir, model_name + "_base_fold" + str(fold_no) + ".h5")
    
    # Increase fold number
    fold_no = fold_no + 1
    
    model.save(model_split_save_path, include_optimizer=False)

    del model

    gc.collect()

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')

for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')

print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

save_plot_accuracy_loss(history, model_name, model_save_dir, 1)